In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# 数据集的原始路径
dataset_path = 'D:/tester01'

train_path = 'D:/tester01/train'
val_path = 'D:/tester01/val'
os.makedirs(train_path, exist_ok=True)
os.makedirs(val_path, exist_ok=True)

# 分类别进行划分
categories = ['cataract', 'normal', 'surgery']

for category in categories:
    # 创建类别的训练和验证目录
    os.makedirs(os.path.join(train_path, category), exist_ok=True)
    os.makedirs(os.path.join(val_path, category), exist_ok=True)

    # 获取所有图片路径
    images = os.listdir(os.path.join(dataset_path, category))
    images = [os.path.join(dataset_path, category, image) for image in images]

    # 划分数据集
    train_images, val_images = train_test_split(images, test_size=0.2) 

    # 移动图片到对应的训练和验证目录
    for image in train_images:
        shutil.move(image, os.path.join(train_path, category))

    for image in val_images:
        shutil.move(image, os.path.join(val_path, category))

# 加载预训练的InceptionV3模型
base_model = InceptionV3(weights='imagenet', include_top=False)

# 添加自定义层
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(3, activation='softmax')(x) 

model = Model(inputs=base_model.input, outputs=predictions)

# 冻结InceptionV3的所有层
for layer in base_model.layers:
    layer.trainable = False

# 编译模型
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# 数据增强
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

# 准备数据集
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(299, 299),
    batch_size=64,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    val_path,
    target_size=(299, 299),
    batch_size=64,
    class_mode='categorical')

# 训练模型
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=5,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size)

In [9]:
# 保存训练好的模型
model.save("model.h5")